## Query coreference
We have previously determined that Google Translate has trouble understanding same-gender (SG) relationships in French, Italian, Spanish. Let's see if that bias persists in coreference resolution in the following format:

- `Juan and Carla talked about her girlfriend`
- `Juan y Carla hablaban sobre su novia`

In [7]:
## does spacy work??
import spacy
nlp = spacy.load('es_core_news_sm')
test_sent = 'Juan y Carla hablaban sobre su novia'
doc = nlp(test_sent)
sents = list(doc.sents)
print(dir(sents[0]))

['_', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_fix_dep_copy', '_vector', '_vector_norm', 'as_doc', 'char_span', 'conjuncts', 'doc', 'end', 'end_char', 'ent_id', 'ent_id_', 'ents', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'kb_id', 'kb_id_', 'label', 'label_', 'lefts', 'lemma_', 'n_lefts', 'n_rights', 'noun_chunks', 'orth_', 'remove_extension', 'rights', 'root', 'sent', 'sentiment', 'set_extension', 'similarity', 'start', 'start_char', 'subtree', 'tensor', 'text', 'text_with_ws', 'to_array', 'vector', 'vector_norm', 'vocab']


In [14]:
print(sents[0].noun_chunks)

[Juan, Carla, su novia]


Next step: try `CoreNLP` via `stanza`, which means setting up a separate server...fun.

In [19]:
from stanza.server import CoreNLPClient
import os
corenlp_dir = '/Users/istewart/Documents/Research/CoreNLP/stanford-corenlp-4.2.2'
os.environ['CORENLP_HOME'] = corenlp_dir

2021-08-21 16:34:19 INFO: Writing properties to tmp file: corenlp_server-bde5ddb2d4354b85.props


In [33]:
test_sent = 'Juan y la Señora Carla hablaban sobre su novia, y ella no sabía qué hacer'
with CoreNLPClient(annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'coref'],  memory='2G', endpoint='http://localhost:9000', be_quiet=True, properties='spanish') as client:
    document = client.annotate(test_sent)

2021-08-21 16:47:03 INFO: Using CoreNLP default properties for: spanish.  Make sure to have spanish models jar (available for download here: https://stanfordnlp.github.io/CoreNLP/) in CLASSPATH
2021-08-21 16:47:03 INFO: Starting server with command: java -Xmx2G -cp /Users/istewart/Documents/Research/CoreNLP/stanford-corenlp-4.2.2/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties spanish -annotators tokenize,ssplit,pos,lemma,ner,coref -preload -outputFormat serialized


In [34]:
display(document)

text: "Juan y la Se\303\261ora Carla hablaban sobre su novia, y ella no sab\303\255a qu\303\251 hacer"
sentence {
  token {
    word: "Juan"
    pos: "PROPN"
    value: "Juan"
    before: ""
    after: " "
    originalText: "Juan"
    ner: "PERSON"
    lemma: "juan"
    beginChar: 0
    endChar: 4
    utterance: 0
    speaker: "PER0"
    tokenBeginIndex: 0
    tokenEndIndex: 1
    hasXmlContext: false
    isNewline: false
    coarseNER: "PERSON"
    fineGrainedNER: "PERSON"
    corefMentionIndex: 0
    entityMentionIndex: 0
    nerLabelProbs: "PERSON=0.9756629291762796"
  }
  token {
    word: "y"
    pos: "CCONJ"
    value: "y"
    before: " "
    after: " "
    originalText: "y"
    ner: "O"
    lemma: "y"
    beginChar: 5
    endChar: 6
    utterance: 0
    speaker: "PER0"
    tokenBeginIndex: 1
    tokenEndIndex: 2
    hasXmlContext: false
    isNewline: false
    coarseNER: "O"
    fineGrainedNER: "O"
    nerLabelProbs: "O=0.9935061997903432"
  }
  token {
    word: "la"
    pos: 

OK! Looks like the coreference resolution here doesn't really work at all.